In [150]:
import pandas as pd
import pickle
import requests
import numpy as np
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import random
from scipy.spatial.distance import cosine
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Concatenate, Input, concatenate, Dropout
from keras.optimizers import Adagrad
from keras import regularizers

In [2]:
perturbagen_class = pickle.load(open("../data/all_perturbagen_class.pkl", "rb"))
perturbagens = list(perturbagen_class.keys())

In [3]:
df = pd.read_csv("../data/full_geneexp_phase2_1004.csv")
def getcellline(x):
    cellline = x.split("_")[1]
    return cellline
df['celline'] = df['signature'].apply(getcellline)

In [4]:
df["atc_level_one"] = [val[0] for val in list(df["atc"])]
imp_columns = ['target', 'atc', 'celline', 'atc_level_one']+list(df.columns[7:985])
df = df[imp_columns]

In [5]:
df.head()

,target,atc,celline,atc_level_one,780,7849,6193,23,9552,387,...,54681,11000,6915,6253,7264,5467,2767,23038,57048,79716
0,BRD-K70330367,N04BB01,YAPC,N,-1.153825,-0.050292,-0.517787,0.244794,-0.399110,0.317308,...,0.440725,0.115513,-0.282635,0.666874,-0.319930,-0.131123,-0.213662,-1.007887,-0.182830,0.524592
1,BRD-K70330367,N04BB01,YAPC,N,-0.150379,0.557618,-0.106715,-0.680913,-0.356638,0.821385,...,-0.017656,2.434079,-0.580103,-0.610206,0.188383,-0.309202,0.056737,-0.003205,-0.434692,-0.177766
2,BRD-K70330367,N04BB01,HA1E,N,0.038450,0.620755,-0.324466,0.136990,0.141210,-0.112033,...,0.224989,0.076725,-0.925917,0.711872,-0.341481,0.437485,-0.052817,-0.376699,-0.047295,-0.215700
3,BRD-K70330367,N04BB01,PC3,N,2.363550,-0.061650,0.691550,0.524950,-0.593000,0.109150,...,0.212650,0.114100,-0.307550,-1.156950,0.508250,-0.230200,-0.314000,-1.208450,-0.630500,-0.447650
4,BRD-K70330367,N04BB01,PC3,N,-0.512800,-0.442350,0.697400,0.108150,-0.552350,0.938150,...,0.358500,-0.641800,-0.603150,-0.792450,0.384800,-0.458000,-0.074800,-0.196750,-0.669750,0.074850


In [18]:
atc_level_vals = np.unique(df.atc_level_one)
cell_lines = [k for k,_ in Counter(df.celline).most_common(7)]

### Dataset Prep

In [19]:
df = df[df.celline.isin(cell_lines)]

le = LabelEncoder()
le.fit(df['atc_level_one'])

df['atc_level_one'] = le.transform(df['atc_level_one'])

pert_train, pert_test = train_test_split(np.unique(df.target), test_size=0.3)

df_train = df[df.target.isin(pert_train)]
df_test = df[df.target.isin(pert_test)]

df_cell_line_train = {}
for cell_line in cell_lines:
    df_cell_line_train[cell_line] = df_train[df_train.celline==cell_line]
    
df_cell_line_test = {}
for cell_line in cell_lines:
    df_cell_line_test[cell_line] = df_test[df_test.celline==cell_line]

In [53]:
oe = OneHotEncoder()
oe.fit(le.transform(atc_level_vals).reshape(len(atc_level_vals), 1))
oe.transform(np.array([0]).reshape(-1, 1)).toarray()

/home/kavya/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

### Base ANN

In [80]:
class network:
    def __init__(self):
        self.model = Sequential()
        self.model.add(Dense(25, activation='selu', input_dim=978))
        self.model.add(Dense(14, activation='softmax'))
        self.model.compile(optimizer='adagrad',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
    def predict(self, X):
        return self.model.predict(X)
    
    def fit(self, X, y):
        self.model.fit(X, y, epochs=20)

In [81]:
network_models = {}

for cell_line in cell_lines:
    X = df_cell_line_train[cell_line][df.columns[4:]]
    y = oe.transform(np.array(df_cell_line_train[cell_line]['atc_level_one']).reshape(-1, 1)).toarray()
    print(y.shape)
    
    network_models[cell_line] = network()
    network_models[cell_line].fit(X, y) 
    
    y_pred = network_models[cell_line].predict(df_cell_line_test[cell_line][df.columns[4:]])
    y_true = oe.transform(np.array(df_cell_line_test[cell_line]['atc_level_one']).reshape(-1, 1)).toarray()
    print(accuracy_score(df_cell_line_test[cell_line]['atc_level_one'], np.argmax(y_pred, axis=1)))


(4614, 14)
Epoch 1/20
4614/4614 [==============================] - 1s 210us/step - loss: 2.4647 - acc: 0.2575
Epoch 2/20
4614/4614 [==============================] - 0s 73us/step - loss: 1.8250 - acc: 0.4094
Epoch 3/20
4614/4614 [==============================] - 0s 74us/step - loss: 1.6418 - acc: 0.4694
Epoch 4/20
4614/4614 [==============================] - 0s 74us/step - loss: 1.5195 - acc: 0.5117
Epoch 5/20
4614/4614 [==============================] - 0s 73us/step - loss: 1.4208 - acc: 0.5485
Epoch 6/20
4614/4614 [==============================] - 0s 73us/step - loss: 1.3427 - acc: 0.5774
Epoch 7/20
4614/4614 [==============================] - 0s 75us/step - loss: 1.2747 - acc: 0.6016
Epoch 8/20
4614/4614 [==============================] - 0s 101us/step - loss: 1.2161 - acc: 0.6207
Epoch 9/20
4614/4614 [==============================] - 0s 85us/step - loss: 1.1653 - acc: 0.6400
Epoch 10/20
4614/4614 [==============================] - 0s 73us/step - loss: 1.1200 - acc: 0.6513
Epoch 

4480/4480 [==============================] - 0s 76us/step - loss: 1.8357 - acc: 0.3984
Epoch 3/20
4480/4480 [==============================] - 0s 72us/step - loss: 1.6628 - acc: 0.4616
Epoch 4/20
4480/4480 [==============================] - 0s 73us/step - loss: 1.5426 - acc: 0.5067
Epoch 5/20
4480/4480 [==============================] - 0s 73us/step - loss: 1.4511 - acc: 0.5335
Epoch 6/20
4480/4480 [==============================] - 0s 77us/step - loss: 1.3729 - acc: 0.5645
Epoch 7/20
4480/4480 [==============================] - 0s 97us/step - loss: 1.3077 - acc: 0.5826
Epoch 8/20
4480/4480 [==============================] - 0s 75us/step - loss: 1.2502 - acc: 0.6056
Epoch 9/20
4480/4480 [==============================] - 0s 85us/step - loss: 1.1992 - acc: 0.6176
Epoch 10/20
4480/4480 [==============================] - 0s 87us/step - loss: 1.1524 - acc: 0.6355
Epoch 11/20
4480/4480 [==============================] - 0s 74us/step - loss: 1.1105 - acc: 0.6525
Epoch 12/20
4480/4480 [======

In [83]:
pred_cell_line = []
pred = []
for pert in pert_test:
    predictions = []
    all_predictions = []
    for cell_line in cell_lines:
        mini_df = df_cell_line_test[cell_line][df_cell_line_test[cell_line]['target']==pert]
        if mini_df.shape[0]==0:
            continue
        answers = []
        for index in mini_df.index:
            answers+=(list(np.argmax(network_models[cell_line].predict(np.array(
                mini_df[df.columns[4:]].loc[index]).reshape(1, -1)), axis=1)))
            all_predictions.append(answers[-1])
        predictions.append(Counter(answers).most_common(1)[0][0])
    pred.append(Counter(all_predictions).most_common(1)[0][0])
    pred_cell_line.append(Counter(predictions).most_common(1)[0][0])

In [84]:
y_true = [np.array(df_test[df_test['target']==pert]['atc_level_one'])[0] for pert in pert_test]
accuracy_score(y_true, pred)

0.3021276595744681

### DenseNet

In [152]:
class denseNetwork:
    def __init__(self, d=6, k=40):
        self.d = d
        self.k = k
        
        _input = Input(shape=(978, ))
        Inputs = [_input]
        for layer in range(d-1):
            _dense = Dense(k, activation='selu', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))(_input)
            _dense = Dropout(rate=0.9)(_dense)
            _input = Input(shape=(978, ))
            Inputs.append(_input)
            _input = concatenate([_dense, _input])

        merge_output = Dense(14, activation='softmax')(_input)

        self.model = Model(inputs=Inputs, outputs=merge_output)
        ada_grad = Adagrad(lr=0.1, epsilon=1e-08, decay=0.0)
        self.model.compile(optimizer=ada_grad, loss='categorical_crossentropy',
                       metrics=['accuracy'])
    
    def predict(self, X):
        return self.model.predict([X for _ in range(self.d)])
    
    def fit(self, X, y):
        self.model.fit([X for _ in range(self.d)], y, epochs=20)

In [154]:
denseNetwork_models = {}

for cell_line in cell_lines:
    X = df_cell_line_train[cell_line][df.columns[4:]]
    y = oe.transform(np.array(df_cell_line_train[cell_line]['atc_level_one']).reshape(-1, 1)).toarray()
    print(y.shape)
    
    denseNetwork_models[cell_line] = denseNetwork(3, 20)
    denseNetwork_models[cell_line].fit(X, y) 
    
    y_pred = denseNetwork_models[cell_line].predict(df_cell_line_test[cell_line][df.columns[4:]])
    y_true = oe.transform(np.array(df_cell_line_test[cell_line]['atc_level_one']).reshape(-1, 1)).toarray()
    print(accuracy_score(df_cell_line_test[cell_line]['atc_level_one'], np.argmax(y_pred, axis=1)))


(4614, 14)
Epoch 1/20
4614/4614 [==============================] - 4s 892us/step - loss: 33.5388 - acc: 0.2510
Epoch 2/20
4614/4614 [==============================] - 1s 150us/step - loss: 24.3940 - acc: 0.4254
Epoch 3/20
4614/4614 [==============================] - 1s 137us/step - loss: 18.3365 - acc: 0.4950
Epoch 4/20
4614/4614 [==============================] - 1s 166us/step - loss: 14.8871 - acc: 0.5429
Epoch 5/20
4614/4614 [==============================] - 1s 164us/step - loss: 12.8940 - acc: 0.5821
Epoch 6/20
4614/4614 [==============================] - 1s 165us/step - loss: 11.5593 - acc: 0.6129
Epoch 7/20
4614/4614 [==============================] - 1s 171us/step - loss: 10.6779 - acc: 0.6335
Epoch 8/20
4614/4614 [==============================] - 1s 166us/step - loss: 9.9816 - acc: 0.6437
Epoch 9/20
4614/4614 [==============================] - 1s 166us/step - loss: 9.4640 - acc: 0.6656
Epoch 10/20
4614/4614 [==============================] - 1s 163us/step - loss: 9.0457 - acc

4480/4480 [==============================] - 4s 982us/step - loss: 33.5029 - acc: 0.2469
Epoch 2/20
4480/4480 [==============================] - 1s 164us/step - loss: 23.8411 - acc: 0.4143
Epoch 3/20
4480/4480 [==============================] - 1s 163us/step - loss: 17.7271 - acc: 0.5047
Epoch 4/20
4480/4480 [==============================] - 1s 163us/step - loss: 14.5995 - acc: 0.5480
Epoch 5/20
4480/4480 [==============================] - 1s 164us/step - loss: 12.7652 - acc: 0.5817
Epoch 6/20
4480/4480 [==============================] - 1s 162us/step - loss: 11.5174 - acc: 0.6161
Epoch 7/20
4480/4480 [==============================] - 1s 163us/step - loss: 10.7262 - acc: 0.6263
Epoch 8/20
4480/4480 [==============================] - 1s 160us/step - loss: 10.0518 - acc: 0.6437
Epoch 9/20
4480/4480 [==============================] - 1s 160us/step - loss: 9.4849 - acc: 0.6621
Epoch 10/20
4480/4480 [==============================] - 1s 162us/step - loss: 9.0827 - acc: 0.6746
Epoch 11/20


In [155]:
pred_cell_line = []
pred = []
for pert in pert_test:
    predictions = []
    all_predictions = []
    for cell_line in cell_lines:
        mini_df = df_cell_line_test[cell_line][df_cell_line_test[cell_line]['target']==pert]
        if mini_df.shape[0]==0:
            continue
        answers = []
        for index in mini_df.index:
            answers+=(list(np.argmax(denseNetwork_models[cell_line].predict(np.array(
                mini_df[df.columns[4:]].loc[index]).reshape(1, -1)), axis=1)))
            all_predictions.append(answers[-1])
        predictions.append(Counter(answers).most_common(1)[0][0])
    pred.append(Counter(all_predictions).most_common(1)[0][0])
    pred_cell_line.append(Counter(predictions).most_common(1)[0][0])

In [156]:
y_true = [np.array(df_test[df_test['target']==pert]['atc_level_one'])[0] for pert in pert_test]
accuracy_score(y_true, pred)

0.2936170212765957